In [1]:
%cd /data/ai_club/team_C
# pip intall spectral --user 
from spectral import *
from PIL import Image, ImageFile
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2

/data/ai_club/team_C


In [2]:
# Loading spectral image
spectral_img = open_image('ov-63-hd-16ca.hdr')

# Instantiating incremental PCA
pca = IncrementalPCA(n_components=3)

In [3]:
# Calculating macro chunk dimensions, row wise, with padding
spectral_shape = spectral_img.shape
macro_chunk_start = int(spectral_shape[0]/2) - 50
macro_chunk_end = ((spectral_shape[0]//10) * 6) + 50
macro_chunk_size = macro_chunk_end - macro_chunk_start

In [4]:
# Standardizing and chunking the spectral image by row and fitting sklearn's partial PCA
grid = []
for y in range(macro_chunk_start, macro_chunk_end):
    if y % 200 == 0:
        print(f"PCA fit: {y}")
    row = spectral_img[y, :, :][0]
    pca.partial_fit(row)
    grid.append(row)

PCA fit: 9000
PCA fit: 9200
PCA fit: 9400
PCA fit: 9600
PCA fit: 9800
PCA fit: 10000
PCA fit: 10200
PCA fit: 10400
PCA fit: 10600


In [5]:
# Reducing the dimensionality of the hyperspectral image to 3 principal components
results = None
for i, row in enumerate(grid):
    if i % 200 == 0:
        print(f"PCA transform: {i}")
    transformed_row = pca.transform(row)
    if results is None:
        results = transformed_row
    else:
        results = np.vstack((results, transformed_row))

PCA transform: 10795
PCA transform: 10795
PCA transform: 10795
PCA transform: 10795
PCA transform: 10795
PCA transform: 10795
PCA transform: 10795
PCA transform: 10795
PCA transform: 10795
PCA transform: 10795


In [6]:
# Reshaping results array
results_t = np.reshape(results, (macro_chunk_size, spectral_shape[1], 3))

In [7]:
PC_2d_Norm = np.zeros((macro_chunk_size, spectral_shape[1], 3))
for i in range(3):
    PC_2d_Norm[:,:,i] = cv2.normalize(results_t[:,:,i], np.zeros((macro_chunk_size, spectral_shape[1], 3)), 0, 255, cv2.NORM_MINMAX)

In [8]:
# Using Pillow to create a new image with RGB mapping to the 3 principal components 
image = Image.new('RGB', (spectral_shape[1], macro_chunk_size))
for y in range(macro_chunk_size):
    for x in range(spectral_shape[1]):
        r, g, b = PC_2d_Norm[y][x]
        image.putpixel((x, y), (np.uint8(r), np.uint8(g), np.uint8(b), 255))

# Saving the image
image.save("Full_PCA.png", "PNG")